# Importing the Required Libraries

In [1]:
pip install openai

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.1/70.1 KB 3.1 MB/s eta 0:00:00


In [2]:
pip install tiktoken

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 39.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.5/73.5 KB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 KB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 81.3 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.25.1
    Uninstalling requests-2.25.1:
      Successfully uninstalled requests-2.25.1


In [3]:
pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 61.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 22.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 66.0 MB/s eta 0:00:00


In [51]:
import openai
import pandas as pd
import numpy as np
import pickle
from transformers import GPT2TokenizerFast
from typing import List
import os
openai.api_key = "key"


# Reading the datasets

Read the csv files that we created using the text files containing text from the Manual

In [5]:
article_1 = pd.read_csv('/content/Articles.csv')
article_2 = pd.read_csv('/content/Articles 2.csv')
chapters = pd.read_csv('/content/Chapters.csv')

In [6]:
df = pd.concat([chapters, article_1, article_2])

In [7]:
heading = []
for title in df["Title"]:
  if title.startswith("Article"):
    heading.append("Article Information")
  else:
    heading.append("Budget Information")

In [8]:
df["Heading"] = heading

In [9]:
df.dropna(inplace = True)

In [10]:
tokenizer = GPT2TokenizerFast.from_pretrained("gpt2")

def count_tokens(text: str):
    """count the number of tokens in a string"""
    return len(tokenizer.encode(text))

In [11]:
df["Tokens"] = df["Text"].apply(count_tokens)

Token indices sequence length is longer than the specified maximum sequence length for this model (1977 > 1024). Running this sequence through the model will result in indexing errors


In [12]:
df

,Title,Text,Heading,Tokens
0,ADMINISTRATIVE MINISTRIES,Chief Accounting Authority of eachMinistry/Dep...,Budget Information,65
1,APPROPRIATION BILL,The Constitution under Article 114 states“no m...,Budget Information,257
2,BUDGET ACTIVITIES AND TIMELINES,In the entire process of Budgetpreparation com...,Budget Information,321
3,Budget Circular,Commencement of Budget processtakes place with...,Budget Information,110
4,BUDGET DOCUMENTS FINANCE MINISTER’S SPEECH:,KEY TO BUDGET DOCUMENTS:(i) Editorial revision...,Budget Information,391
...,...,...,...,...
6,Article 267-,Contingency Fund.. Parliament may by law estab...,Article Information,147
7,Article 275-,Grants from the Union to certainStates.. Such ...,Article Information,100
8,Article 280-,"Finance Commission.. The President shall, with...",Article Information,288
9,Article 281-,Recommendations of the FinanceCommission.. The...,Article Information,53


In [13]:
df.shape

(135, 4)

In [14]:
df = df.set_index(["Title", "Heading"])

In [15]:
df.to_csv("All.csv",index=False)

# Embedding

Used the openai cookbook for the codes below

In [28]:
COMPLETIONS_MODEL = "text-davinci-003"
MODEL_NAME = "davinci"
DOC_EMBEDDINGS_MODEL = "text-search-davinci-doc-001"
QUERY_EMBEDDINGS_MODEL = "text-search-davinci-query-001"

In [17]:
def get_embedding(text: str,model):
    result = openai.Embedding.create(
      input=text,model=model)
    return result["data"][0]["embedding"]

def get_doc_embedding(text: str):
    return get_embedding(text, DOC_EMBEDDINGS_MODEL)

def get_query_embedding(text: str):
    return get_embedding(text, QUERY_EMBEDDINGS_MODEL)

def compute_doc_embeddings(df: pd.DataFrame):
    """
    Create an embedding for each row in the dataframe using the OpenAI Embeddings API.
    
    Return a dictionary that maps between each embedding vector and the index of the row that it corresponds to.
    """
    return {idx: get_doc_embedding(r.Text) for idx, r in df.iterrows()}

In [18]:
def load_embeddings(fname: str):
    """
    Read the document embeddings and their keys from a CSV.
    
    fname is the path to a CSV with exactly these named columns: 
        "title", "heading", "0", "1", ... up to the length of the embedding vectors.
    """
    
    df = pd.read_csv(fname, header=0)
    max_dim = max([int(c) for c in df.columns if c != "Title" and c != "Heading"])
    return {
           (r.Title, r.Heading): [r[str(i)] for i in range(max_dim + 1)] for _, r in df.iterrows()
    }

Splitting the dataset to prevent rate limit reached error

In [38]:
df1 = df.iloc[:30,:]
df2 = df.iloc[30:60,:]
df3 = df.iloc[60:90,:]
df4 = df.iloc[90:,:]

Ran the 4 compute_doc_embeddings after a minute gap

In [31]:
context_embeddings1 = compute_doc_embeddings(df1)

In [34]:
context_embeddings2 = compute_doc_embeddings(df2)

In [35]:
context_embeddings3 = compute_doc_embeddings(df3)

In [37]:
context_embeddings4 = compute_doc_embeddings(df4)

Merge all the dictionaries to get the entire dataset back

In [39]:
def Merge(dict1, dict2):
    res = {**dict1, **dict2}
    return res

d = Merge(context_embeddings1,context_embeddings2)
d = Merge(d,context_embeddings3)
embeddings_df = Merge(d,context_embeddings4)

In [40]:
document_embeddings = embeddings_df

In [41]:
def vector_similarity(x: List[float], y: List[float]):
    """
    We could use cosine similarity or dot product to calculate the similarity between vectors.
    In practice, we have found it makes little difference. 
    """
    return np.dot(np.array(x), np.array(y))

In [42]:
def order_document_sections_by_query_similarity(query: str, contexts: dict):
    """
    Find the query embedding for the supplied query, and compare it against all of the pre-calculated document embeddings
    to find the most relevant sections. 
    
    Return the list of document sections, sorted by relevance in descending order.
    """
    query_embedding = get_query_embedding(query)
    
    document_similarities = sorted([
        (vector_similarity(query_embedding, doc_embedding), doc_index) for doc_index, doc_embedding in contexts.items()
    ], reverse=True)
    
    return document_similarities

In [48]:
order_document_sections_by_query_similarity("What is Article 22?", document_embeddings)[:5]

[(0.28741960079574247, ('Article 13.', 'Article Information')),
 (0.2662641643644913, ('Article 239AB.', 'Article Information')),
 (0.257630055868165, ('Article 357.', 'Article Information')),
 (0.257472113880603, ('Article 115.', 'Article Information')),
 (0.2565524192981189, ('Article 271.', 'Article Information'))]

In [44]:
MAX_SECTION_LEN = 1500
SEPARATOR = "\n* "

tokenizer = GPT2TokenizerFast.from_pretrained("gpt2")
separator_len = len(tokenizer.tokenize(SEPARATOR))

f"Context separator contains {separator_len} tokens"

'Context separator contains 3 tokens'

In [45]:
def construct_prompt(question: str, context_embeddings: dict, df: pd.DataFrame):
    """
    Fetch relevant 
    """
    most_relevant_document_sections = order_document_sections_by_query_similarity(question, context_embeddings)
    
    chosen_sections = []
    chosen_sections_len = 0
    chosen_sections_indexes = []
     
    for _, section_index in most_relevant_document_sections:
        # Add contexts until we run out of space.        
        document_section = df.loc[section_index]
        
        chosen_sections_len += document_section.Tokens + separator_len
        if chosen_sections_len > MAX_SECTION_LEN:
            break
            
        chosen_sections.append(SEPARATOR + document_section.Text.replace("\n", " "))
        chosen_sections_indexes.append(str(section_index))
            
    # Useful diagnostic information
    print(f"Selected {len(chosen_sections)} document sections:")
    print("\n".join(chosen_sections_indexes))
    
    header = """Answer the question as truthfully as possible using the provided context, and if the answer is not contained within the text below, say "I don't know."\n\nContext:\n"""
    
    return header + "".join(chosen_sections) + "\n\n Q: " + question + "\n A:"

In [46]:
COMPLETIONS_API_PARAMS = {
    # We use temperature of 0.0 because it gives the most predictable, factual answer.
    "temperature": 0.0,
    "max_tokens": 300,
    "model": COMPLETIONS_MODEL,
}

In [47]:
def answer_query_with_context(
    query: str,
    df: pd.DataFrame,
    document_embeddings,show_prompt: bool = False):
    prompt = construct_prompt(
        query,
        document_embeddings,
        df
    )
    
    if show_prompt:
        print(prompt)

    response = openai.Completion.create(
                prompt=prompt,
                **COMPLETIONS_API_PARAMS
            )

    return response["choices"][0]["text"].strip(" \n")

In [52]:
answer_query_with_context("What is Article 13?", df, document_embeddings)

Selected 8 document sections:
('Article 13.', 'Article Information')
('Article 114.', 'Article Information')
('Article 117.', 'Article Information')
('Article 271.', 'Article Information')
('Article 117-', 'Article Information')
('Article 267-', 'Article Information')
('Article 115.', 'Article Information')
('Article 357.', 'Article Information')


"I don't know."

In [53]:
query = "What is Budget in Indian Budget 2023?"
answer = answer_query_with_context(query, df, document_embeddings)

print(f"\nQ: {query}\nA: {answer}")

Selected 5 document sections:
('OUTCOME-OUTPUT MONITORINGFRAMEWORK (OOMF)', 'Budget Information')
('Budget', 'Budget Information')
('FINANCE COMMISSION', 'Budget Information')
('FEATURES OF BUDGET', 'Budget Information')
('Scope of Budget', 'Budget Information')

Q: What is Budget in Indian Budget 2023?
A: Budget in Indian Budget 2023 is a statement of the estimated receipts and expenditure of the Government for the financial year. It is presented to the Parliament each year on the 1st February or any other suitable date by the Finance Minister.


In [54]:
query = "Tell me about the Authorisation of expenditure by President.?"
answer = answer_query_with_context(query, df, document_embeddings)

print(f"\nQ: {query}\nA: {answer}")

Selected 6 document sections:
('Article 74.', 'Article Information')
('Article 115.', 'Article Information')
('Excess Grants', 'Budget Information')
('CHECK LIST FOR BUDGET EXECUTION', 'Budget Information')
('APPROPRIATION BILL', 'Budget Information')
('Article 30.', 'Article Information')

Q: Tell me about the Authorisation of expenditure by President.?
A: Where the Legislative Assembly is dissolved, or its functioning as such Assembly remains suspended, on account of an order under section 73, it shall be competent for the President to authorise, when the House of the People is not in Session, expenditure from the Consolidated Fund of the Union territory of Jammu and Kashmir pending the sanction of such expenditure by Parliament.


In [55]:
while query!=0:
  query = input('Question')
  answer = answer_query_with_context(query, df, document_embeddings)
  print(f"\nQ: {query}\nA: {answer}")

QuestionWhat article entails Jammu Kashmir
Selected 6 document sections:
('Article 13.', 'Article Information')
('Article 42.', 'Article Information')
('Article 239AB.', 'Article Information')
('Article 273.', 'Article Information')
('Article 74.', 'Article Information')
('Article 50.', 'Article Information')

Q: What article entails Jammu Kashmir
A: Article 239A of the Constitution entails Jammu and Kashmir.
QuestionWhat is article 22
Selected 7 document sections:
('Article 13.', 'Article Information')
('Article 239AB.', 'Article Information')
('Article 271.', 'Article Information')
('Article 357.', 'Article Information')
('Article 115.', 'Article Information')
('Article 239', 'Article Information')
('Article 117-', 'Article Information')

Q: What is article 22
A: I don't know.
QuestionWhat is Finance Bill
Selected 4 document sections:
('FINANCE BILL', 'Budget Information')
('SUMMARY FOR THE CABINET', 'Budget Information')
('Article 117.', 'Article Information')
('Article 110.', 'Arti

KeyboardInterrupt: ignored